## Homework 4 for ECON-GA 3002, Section 12.

**{Name, N-Number, Email}**: {Arnav Sood, N11193569, asood@nyu.edu}

**Overview**: Show the downward bias in the standard OLS estimate of the lag coefficient by running a large number of trials for a few (alpha, n) pairs, and then plotting. Optimize for speed.

**Disclosure**: Was not written using vim (sorry). Was instead written using a Jupyter notebook and REPL.

In [1]:
# Vanilla Python version. Before optimizing, get a vanilla code set in Python.

# Update: parallelization is hard.

# The optimization path will probably look something like this:
'''
    - Write vanilla Python code.
            - Wall Time: 2m41s
    - Make easy Python improvements (e.g, @jit, numbafy)
            - @jit: Wall Time: 17.9s
            - @jit on Mac (no VM): 11.3s
            - add numpy: 2s?
            - remove print: like 5s?
    - Make harder Python improvements (not sure what they are a priori)
    - Write code in C if it makes sense
    - Figure out what PEP8 is, and make sure I'm using it
'''

# Throw in numba jit compile.
from numba import jit

# Add more numpy stuff.
import numpy as np

# Setup variables.
alphas = np.arange(0.5,1,0.1)
ns = np.arange(50,550,50)

# Define a function to take an (alpha, n, beta) tuple, and return the bias for the estimate.

# Store this as a lambda function so it can compile the operation once. (?)
estimate = lambda sumx, sumy, sumxsq, sumxy, n : (sumxy - (n-1)*(sumx/(n-1))*(sumy/(n-1)))/(sumxsq - (n-1)*math.pow((sumx/(n-1)),2));

@jit
def bias(alpha, n, beta):
        sumx = 0
        sumy = 0
        sumxy = 0
        sumxsq = 0
        x = rand()*100
        for i in range(n):
            sumx += x
            sumxsq += math.pow(x,2)
            y = beta + alpha*x + rand()
            sumxy += x*y
            sumy += y
            x = y
        val = estimate(sumx, sumy, sumxsq, sumxy, n) - alpha
        return val

# Define a function to take an (alpha, n, beta) tuple, estimate 10,000 times, and then average to get the expected bias.
from numpy.random import normal as rand
import math

@jit
def calcbias(alpha, n, beta):
    biases = np.zeros(10000)
    for i in range(10000):
        biases[i] = bias(alpha,n,beta)
    avg = np.mean(biases)
    return avg

# Define a function to call the above on the right sets of params, and return the biases.
@jit
def main():
    results = np.zeros(50)
    for i in range(5):
        for j in range(10):
            results[10*i+j] = calcbias(alphas[i],ns[j],1)
    print(results)
    return results


In [2]:
%time x = main()

[-0.0068893  -0.00530987 -0.00393713 -0.00302123 -0.00277637 -0.00274908
 -0.00222198 -0.00203061 -0.00190814 -0.0020535  -0.00898535 -0.00548835
 -0.00399966 -0.00385827 -0.00325968 -0.00292754 -0.00269609 -0.00234095
 -0.00239163 -0.00188504 -0.01009127 -0.00591049 -0.00478598 -0.00390847
 -0.00383308 -0.00319646 -0.00268235 -0.00278073 -0.00258312 -0.00237767
 -0.01509515 -0.00741615 -0.00544057 -0.00418215 -0.00353113 -0.00343919
 -0.00324128 -0.00300572 -0.00257797 -0.00265793 -0.01577805 -0.01135966
 -0.00680381 -0.00508683 -0.00447353 -0.0040401  -0.00371721 -0.00329455
 -0.0028915  -0.00283044]
CPU times: user 11.9 s, sys: 38.1 ms, total: 11.9 s
Wall time: 11.9 s


In [5]:
# Plot.
from matplotlib import pyplot as plt
vec1 = x[0:10]
vec2 = x[10:20]
vec3 = x[20:30]
vec4 = x[30:40]
vec5 = x[40:50]

# Plot each of the vectors against 'n'
plt.plot(ns, vec1, 'r-', ns, vec2, 'b-', ns, vec3, 'm-',ns,vec4,'g-',ns,vec5,'k-')
plt.title('Biases vs. Sample Sizes in OLS Estimate of Lag Coefficient')
plt.xlabel('Sample Size')
plt.ylabel('Bias (Absolute Difference)')
plt.show()

![title](figure_1.png)

## What We See

There are a few properties of the OLS estimator which we can extract from the figure:

**Consistency**: There is a clear asymptotic trend to 0 bias. This is because the OLS estimator is a consistent one, which means that it has exactly this property.

**Downward Bias**: All the data points are negative, which confirms our supposition that the OLS estimator is downward-biased.

**Proportionality**: This isn't a formal statistical term, but roughly speaking, we see that the magnitude of the bias increases with the absolute value of alpha.

**Convergence Rate**: We see a logarithmic-type convergence rate, which implies diminishing returns to additional data points, and a clear steep portion to avoid when running simulation.